### Year 1 Topology - Limit Checking - Overloading, Voltage limits

In [1]:
import pandas as pd
import altair as alt

##### Loading

##### Percentage Loading in Branches 

Calculation related can be found in PSSE PAGVI Manual Chapter 6 Power system network simulations - Section 6.6.7 Activity Rate 

%I = MVA(actual)/(MVA(rated)*Vpu)

Vpu -> Bus Voltage at from end 

In [2]:
list_all = []
list_gens = [0,50,100]
list_lsc = ['lls','rls','hls']
for gen in list_gens:
    for lsc in list_lsc:
        file_csv_out = 'savnw_sol_' + str(gen) +'_'+ lsc + '_loading.csv'
        data_load = pd.read_csv(file_csv_out,header=1, skiprows=0)
        # Extracted data contains branch loading from the metered and non-metered ends. The data is filtered to obtain the branch loading from the metered end.
        data_fil = data_load
        data_fil['From Bus V'] = data_fil['From Bus Ext'].str.extract(r'(\d+\.\d+)') 
        data_fil['To Bus V'] = data_fil['To Bus Ext'].str.extract(r'(\d+\.\d+)')
        data_fil['Scenario']= 'Solar = ' + str(gen) + ' MW, ' + lsc.upper()
        list_all.append(data_fil)
data_all = pd.concat(list_all).reset_index(drop=True)

##### Line and Transformer Loading

Transformer and branch loading are categorised to following brackets based on the range to which it falls into 
* Loading less than 25%
* Loading 25% to 50%
* Loading 50% to 70%
* Loading 70% to 80%
* Loading 80% to 85%
* Loading 85% to 90%
* Loading 90% to 95%
* Loading 95% to 100%
* Loading 100% and above

Separating transformer and line branch loadings

In [3]:
transformer_loading = data_all[data_all['From Bus V']!=data_all['To Bus V']].reset_index(drop=True)
branch_loading = data_all[data_all['From Bus V'] == data_all['To Bus V']].reset_index(drop=True)

Defining loading brackets for transformer and line branches 

In [4]:
data = transformer_loading[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch Flow MVAR','Branch MVA', 'Rate','%MVA']]
data['Loading'] = ''
data.loc[(data['%MVA'] >100), 'Loading'] = '%MVA>100'
data.loc[(data['%MVA'] <100), 'Loading'] = '95<%MVA<100'
data.loc[(data['%MVA'] <95), 'Loading'] = '90<%MVA<95'
data.loc[(data['%MVA'] <90), 'Loading'] = '85<%MVA<90'
data.loc[(data['%MVA'] <85), 'Loading'] = '80<%MVA<85'
data.loc[(data['%MVA'] <80), 'Loading'] = '70<%MVA<80'
data.loc[(data['%MVA'] <70), 'Loading'] = '50<%MVA<70'
data.loc[(data['%MVA'] <50), 'Loading'] = '25<%MVA<50'
data.loc[(data['%MVA'] <25), 'Loading'] = '%MVA<25'

<ipython-input-4-7f16dd71236d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Loading'] = ''


In [5]:
data_l = branch_loading[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch MVA', 'Rate','%I']]
data_l['Loading'] = ''
data_l.loc[(data_l['%I'] >100), 'Loading'] = '%I>100'
data_l.loc[(data_l['%I'] <100), 'Loading'] = '95<%I<100'
data_l.loc[(data_l['%I'] <95), 'Loading'] = '90<%I<95'
data_l.loc[(data_l['%I'] <90), 'Loading'] = '85<%I<90'
data_l.loc[(data_l['%I'] <85), 'Loading'] = '80<%I<85'
data_l.loc[(data_l['%I'] <80), 'Loading'] = '70<%I<80'
data_l.loc[(data_l['%I'] <70), 'Loading'] = '50<%I<70'
data_l.loc[(data_l['%I'] <50), 'Loading'] = '25<%I<50'
data_l.loc[(data_l['%I'] <25), 'Loading'] = '%I<25'

<ipython-input-5-abcf08d21c83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_l['Loading'] = ''


##### Histogram Transformer Branch Loading 

A histogram for each scenario is plotted to check the no of transformer branches in the above defined loading bracket 

In [6]:
alt.Chart(data).mark_bar().encode(
    alt.Y('Loading',scale = alt.Scale(domain=['%MVA>100','95<%MVA<100','90<%MVA<95','85<%MVA<90','80<%MVA<85','70<%MVA<80','50<%MVA<70','25<%MVA<50','%MVA<25'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=3).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that for most of scenarios the transformer branch loading falls into a bracket of loading between 50 to 70 percentage. 

##### Histogram Line Branch Loading 

A histogram is plotted to check the no of line branches in the above defined loading bracket 

In [7]:
alt.Chart(data_l).mark_bar().encode(
    alt.Y('Loading', scale =alt.Scale(domain=['%I>100','95<%I<100','90<%I<95','85<%I<90','80<%I<85','70<%I<80','50<%I<70','25<%I<50','%I<25'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=3).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that for most of scenarios the line branch loading falls into a bracket of loading between 25% to 50% 

In the following section the lines and transformer branches with loading greater than 80% is filtered 

##### Checking transformer loading greater than 80% 

In [8]:
transformer_loading = data_all[data_all['From Bus V']!=data_all['To Bus V']].reset_index(drop=True)
transformer_overload  = transformer_loading[transformer_loading['%MVA'] > 80.0].reset_index(drop=True)
transformer_overload['Branch'] = transformer_overload['From Bus'].astype(str) + ' ' + transformer_overload['From Bus Ext'] + ' to ' + \
transformer_overload['To Bus'].astype(str) + ' '+ transformer_overload['To Bus Ext'] + ' ' + transformer_overload['Ckt ID'].astype(str)
list_branch = transformer_overload['Branch'].unique()
print(list_branch)

['3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1']


In [9]:
transformer_overload

,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Scenario,Branch
0,3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4760,90.0,80.0,120.42,0.03,11.00,150.0,75.85,80.28,13.800,230.00,"Solar = 0 MW, LLS",3018 CATDOG_G 13.800 to 3008 CATDOG 23...
1,3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4775,90.0,80.0,120.42,0.03,11.07,150.0,76.09,80.28,13.800,230.00,"Solar = 0 MW, RLS",3018 CATDOG_G 13.800 to 3008 CATDOG 23...
2,3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4787,90.0,80.0,120.42,0.03,11.13,150.0,76.28,80.28,13.800,230.00,"Solar = 0 MW, HLS",3018 CATDOG_G 13.800 to 3008 CATDOG 23...
3,3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4757,90.0,80.0,120.42,0.03,10.99,150.0,75.81,80.28,13.800,230.00,"Solar = 50 MW, LLS",3018 CATDOG_G 13.800 to 3008 CATDOG 23...
4,3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4772,90.0,80.0,120.42,0.03,11.06,150.0,76.05,80.28,13.800,230.00,"Solar = 50 MW, RLS",3018 CATDOG_G 13.800 to 3008 CATDOG 23...
5,3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4784,90.0,80.0,120.42,0.03,11.12,150.0,76.24,80.28,13.800,230.00,"Solar = 50 MW, HLS",3018 CATDOG_G 13.800 to 3008 CATDOG 23...
6,3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4756,90.0,80.0,120.42,0.03,10.98,150.0,75.79,80.28,13.800,230.00,"Solar = 100 MW, LLS",3018 CATDOG_G 13.800 to 3008 CATDOG 23...
7,3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4770,90.0,80.0,120.42,0.03,11.05,150.0,76.02,80.28,13.800,230.00,"Solar = 100 MW, RLS",3018 CATDOG_G 13.800 to 3008 CATDOG 23...
8,3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4782,90.0,80.0,120.42,0.03,11.11,150.0,76.21,80.28,13.800,230.00,"Solar = 100 MW, HLS",3018 CATDOG_G 13.800 to 3008 CATDOG 23...


##### Checking line branch loading greater than 80% 

In [10]:
branch_loading = data_all[data_all['From Bus V'] == data_all['To Bus V']].reset_index(drop=True)
branch_overload = branch_loading[branch_loading['%I'] > 80.0].reset_index(drop=True)
branch_overload['Branch'] = branch_overload['From Bus'].astype(str) + ' ' + branch_overload['From Bus Ext'] + ' to ' + \
branch_overload['To Bus'].astype(str) + ' '+ branch_overload['To Bus Ext'] + ' ' + branch_overload['Ckt ID'].astype(str)
list_branch = branch_overload['Branch'].unique()
print(list_branch)

[]


In [11]:
branch_overload

,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Scenario,Branch


##### Bus Voltage

Similar to branch loading categorization into brackets of loadings, voltages are also categorised into brackets

In [12]:
list_volt = []
list_gens = [0,50,100]
list_lsc = ['lls','rls','hls']
for gen in list_gens:
    for lsc in list_lsc:
        file_csv_out = 'savnw_sol_' + str(gen) +'_'+ lsc + '_volt.csv'
        data_volt = pd.read_csv(file_csv_out,header=1, skiprows=0)
        data_volt['Scenario']= 'Solar = ' + str(gen) + ' MW, ' + lsc.upper()
        list_volt.append(data_volt)
data_all_volt = pd.concat(list_volt).reset_index(drop=True)
data_all_volt = data_all_volt[['Scenario','Bus Number', 'Bus Name', 'Area', 'Base Voltage', 'Bus Voltage(PU)','Bus Voltage(kV)']]

In [13]:
data_v = data_all_volt
data_v['Voltage Range'] = ''
data_v.loc[(data_v['Bus Voltage(PU)'] >1.1), 'Voltage Range'] = 'Vbus>1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.1), 'Voltage Range'] = '1.05<Vbus<1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.05), 'Voltage Range'] = '1.0<Vbus<1.05'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.0), 'Voltage Range'] = '0.95<Vbus<1.0'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.95), 'Voltage Range'] = '0.95<Vbus<0.9'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.9), 'Voltage Range'] = 'Vbus<0.9'

##### Histogram Bus Voltages

A histogram is plotted to check the no of Buses in each of the above defined brackets 

In [14]:
alt.Chart(data_v).mark_bar().encode(
    alt.Y('Voltage Range', scale =alt.Scale(domain=['Vbus>1.1','1.05<Vbus<1.1','1.0<Vbus<1.05','0.95<Vbus<1.0','0.95<Vbus<0.9','Vbus<0.9'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=3).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that most of the buses have voltages in the bracket of 1.0 PU and 1.05 PU

In the following section, the buses out of upper and lower normal operating range is filtered and reported 

##### Voltage Upper Limits 

In [15]:
list_volt = list(data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)['Bus Number'].unique())
list_volt

[201, 3018, 152]

In [17]:
data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range
0,"Solar = 0 MW, LLS",201,HYDRO,2,500.0,1.059,529.269,1.05<Vbus<1.1
1,"Solar = 0 MW, LLS",3018,CATDOG_G,5,13.8,1.058,14.605,1.05<Vbus<1.1
2,"Solar = 0 MW, RLS",201,HYDRO,2,500.0,1.056,528.058,1.05<Vbus<1.1
3,"Solar = 0 MW, RLS",3018,CATDOG_G,5,13.8,1.055,14.559,1.05<Vbus<1.1
4,"Solar = 0 MW, HLS",201,HYDRO,2,500.0,1.054,527.084,1.05<Vbus<1.1
5,"Solar = 0 MW, HLS",3018,CATDOG_G,5,13.8,1.052,14.523,1.05<Vbus<1.1
6,"Solar = 50 MW, LLS",201,HYDRO,2,500.0,1.060,529.935,1.05<Vbus<1.1
7,"Solar = 50 MW, LLS",3018,CATDOG_G,5,13.8,1.059,14.612,1.05<Vbus<1.1
8,"Solar = 50 MW, RLS",201,HYDRO,2,500.0,1.058,528.765,1.05<Vbus<1.1
9,"Solar = 50 MW, RLS",3018,CATDOG_G,5,13.8,1.056,14.567,1.05<Vbus<1.1


##### Voltage Lower Limits

In [18]:
data_all_volt[data_all_volt['Bus Voltage(PU)']<0.95]['Bus Number'].unique()

array([], dtype=int64)

In [19]:
data_all_volt[data_all_volt['Bus Voltage(PU)']<0.95]

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range


#### Conclusion 

Year 1 Topology limit violation check for overloading and voltage was carried out for the three load scenarios. It was seen that
1. No overloading was reported for any branches
2. The transformer branches for considered solar output and load scenarios reported an overload > 80%
    * 3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1
3. The buses violating the upper voltage limit for various scenarios are:
    * Solar = 0 MW, Scenarios = LLS, RLS, HLS
      BUS 201, 308
    * Solar = 50 MW, Scenarios = LLS, RLS, HLS
      BUS 201, 308
    * Solar = 100 MW, Scenarios = LLS
      BUS 152, 201, 308
    * Solar = 100 MW, Scenarios = RLS, HLS
      BUS 201, 308